### Este notebook possui as seguintes finalidades

1 - Consultar os arquivos da pasta Staging no Google Drive.
2 - Pegar todos os arquivos e transformar em DF.
3 - Retornar os cabeçalhos.
4 - Gerar CSV final
5 - Armazenar os arquivos .dat nas suas pastas corretas

In [34]:
import pandas as pd
# Acessar sistema operacional
import os
# Irá retornar uma lista de arquivos
import glob
# Manipulação de arquivos e diretórios
import shutil

In [55]:
# Onde esses arquivos serão armazenados
pasta_Staging = r'I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\Staging'
# Arquivos de 1 minuto
pasta_Gdrive_1m = r'I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\T1_Meteo_1m_2023\Ascii'
# Arquivos de 30 minutos
pasta_Gdrive_30m = r'I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\T1_meteo_Media_30m_2023\Ascii'
# Caminho csv
arquivo_csv = r'I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\T1_Meteo_1m_2023.csv'

In [56]:
######## Arquivos de 1 Minuto

In [57]:
# Obter a lista de arquivos .dat na pasta 1 minuto
arquivos_staging = glob.glob(f'{pasta_Staging}\\*.dat')

# Se não houve nenhum arquivo
if not arquivos_staging:
    print('Erro: A pasta está vazia.')
    exit() # Terminar o programa.

In [59]:
# Há algumas linhas com erro no CSV, irei pula-las e relatar 
linhas_problematicas = []

try:
    df = pd.read_csv(arquivo_csv,error_bad_lines=False)
except pd.errors.ParserErros as e:
    linhas_com_erros = [int(line.split()[0]) for line in str(e).split('\n')[1:-1]]
    
    # Adicione os índices das linhas problemáticas à lista linhas_problematicas
    linhas_problematicas.extend(linhas_com_erros)
    
print("Índices das linhas problemáticas:", linhas_problematicas)

C:\Users\BlueShift\AppData\Local\Temp\ipykernel_56436\48561845.py:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(arquivo_csv,error_bad_lines=False)
Skipping line 18935: expected 1 fields, saw 2
Skipping line 35030: expected 1 fields, saw 2
Skipping line 59221: expected 1 fields, saw 2
Skipping line 63759: expected 1 fields, saw 2
Skipping line 115576: expected 1 fields, saw 2
Skipping line 142942: expected 1 fields, saw 2
Skipping line 145721: expected 1 fields, saw 2
Skipping line 146763: expected 1 fields, saw 2
Skipping line 147231: expected 1 fields, saw 2
Skipping line 148295: expected 1 fields, saw 2
Skipping line 149697: expected 1 fields, saw 2
Skipping line 155238: expected 1 fields, saw 2
Skipping line 155609: expected 1 fields, saw 2
Skipping line 155728: expected 1 fields, saw 2
Skipping line 155821: expected 1 fields, saw 2
Skipping line 157332: expected 1 fiel

Índices das linhas problemáticas: []


In [38]:
# Criação da função que aceita os dois parâmetros
def test_all_col_exists(df,cols):
# Compreensão de lista que percorre cada elemento da lista
    missing_cols = [col for col in cols if col not in df.columns]
    
    # O print irá trazer as colunas que não estão presentes no DF
    if missing_cols:
        print(f'A seguintes colunas não foram encontradas no {df}: {missing_cols}')
    else:
        print('Todas as colunas foram encontradas no DF')
        
# Colunas para verificação 
colunas_verificadas = [ 'TIMESTAMP', 'RECORD', 'Batt_volt_Min', 'PTemp', 'WindSpeed_1', 'WindDir_1', 'WindSpeed_2', 'WindDir_2',
    'WindSpeed_3', 'WindDir_3', 'WindSpeed_4', 'WindDir_4', 'temp_1_Avg', 'temp_2_Avg', 'temp_3_Avg', 'temp_4_Avg',
    'humid_1_Avg', 'humid_2_Avg', 'humid_3_Avg', 'humid_4_Avg', 'par1_in_Avg', 'par2_in_Avg', 'par3_in_Avg',
    'par4_in_Avg', 'chuva_Tot', 'SBTempC_1_Avg', 'TargTempC_1_Avg', 'SBTempC_2_Avg', 'TargTempC_2_Avg',
    'SBTempC_3_Avg', 'TargTempC_3_Avg', 'SBTempC_4_Avg', 'TargTempC_4_Avg', 'pressao_Avg', 'rad_total_Avg',
    'rad_diff_Avg']

In [42]:
# Variável para armazenar parte do nome do arquivo
meteo_1m = "CR6_T1_meteo_"
arquivos_verificados = []
for arquivo in arquivos_staging:
    # Retornando apenas o nome do arquivo sem o caminho da pasta
    nome_arquivo = os.path.basename(arquivo)
    if meteo_1m in nome_arquivo:
        df = pd.read_csv(arquivo, sep=',',skiprows = [0,2,3]) #header=1)
        # Chama a função para verificar a presença das colunas
        test_all_col_exists(df, colunas_verificadas)
        arquivos_verificados.append(nome_arquivo)
print(f'{len(arquivos_verificados)} arquivos foram verificados')

Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas

In [ ]:
for arquivos_verificados in arquivos_staging:
    df = pd.read_csv(arquivo, sep=',',skiprows = [0,2,3]) #header=1)
    

In [ ]:
# Lista que armazena a quantidade de itens enviados
qntd_enviada = []

for arquivos_verificados in arquivos_staging:
    # Buscar o caminho desse arquivo na staging
    caminho_arquivo_staging = f'{pasta_Staging}\\{arquivos_verificados}'
    # Buscar a pasta Staging no GDrive
    caminho_arquivo_gdrive = f'{pasta_Gdrive_1m}\\{nome_arquivo}'
    # Copiar o arquivo de uma pasta para outra
    shutil.copy(caminho_arquivo_one, caminho_arquivo_gdrive)
    # Para cada arquivo 1 mensagem
    print(f'Arquivo {nome_arquivo} copiado para Gdrive')
    # Adicionar +1 para a lista
    qntd_enviada.append(+1)

In [ ]:
# Chama a função para verificar a presença das colunas
#test_all_col_exists(df, colunas_verificadas)